In [ ]:
%pip install -r requirements.txt

In [2]:
import os
import numpy as np
import time
import constructivo as constr
from auxiliar_functions import *
from local_search import *

#utilizar xlwt para escribir en excel


In [3]:
directory_path = 'JSSP_Instances'
directory_files = os.listdir(directory_path)
directory_files.sort()

In [4]:
import xlrd, xlwt
from xlutils.copy import copy as xl_copy
from xlwt import Workbook


def write_xlsx(machine_order, makespan, time, name_sheet, name_file):
    # Create a new workbook
    
    try:
        # open existing workbook
        rb = xlrd.open_workbook(name_file, formatting_info=True)
        # make a copy of it
        workbook = xl_copy(rb)
    except:
        workbook = Workbook()
    sheet = workbook.add_sheet(name_sheet)
    
    # Iterate over the matrix and write its values to the sheet
    for row_index, row in enumerate(machine_order, start=1):
        for col_index, value in enumerate(row, start=1):
            sheet.write(row_index - 1, col_index - 1, value)  # Note: row_index and col_index should start from 0
    
    # Write makespan and time to the next row
    row_index = len(machine_order)
    sheet.write(row_index, 0, int(makespan))
    sheet.write(row_index, 1, time)

    # Save the workbook to the specified file
    workbook.save(name_file)

In [5]:
with open('cotas.txt', 'r') as f:
    cotas = f.readlines()
    cotas = [int(line.strip()) for line in cotas]

In [8]:
#Reading all the information
for file_num in range(1, len(directory_files)+1):
  file = "JSSP" + str(file_num) + ".txt"
  with open(directory_path + "/" + file, "r" ) as f:
  #with open("../AAA_test", "r" ) as f:
    size = f.readline().split()
    # n = trabajos, m = maquinas
    n, m = int(size[0]), int(size[1])
    data = list(f)
    for i,line in enumerate(data):
      data[i] = [int(x) for x in line.strip().split() if x is not None and x != '']

    processing_time = data[0:n]
    machines_required = data[n:2*n]
  
  # Cota inferior
  machines = {i:[[ j+1, processing_time[j][z], z, False] for z in range(m) for j in range(n)  if machines_required[j][z] == i]for i in range(1,m+1)}
  cota_inferior = cotas[file_num-1]

  # Create job objects
  jobs_constr = [constr.Job(np.array(machines_required)[i, :], np.array(processing_time)[i, :], i + 1) for i in range(n)]

  #Sorted array in which the first value is always the first machine that 
  machines_constr = [constr.Machine(i+1) for i in range(m)]
  # Record the start time
  constr_start_time = time.time(); ans_constr, machine_order_constr = constr.constructivo(jobs_constr, machines_constr); constr_end_time = time.time()

  rvns_sol = reduced_neighborhood_search(machine_order_constr, processing_time, machines_required, n, m, times_to_test=1000)

  first_swap_start_time = time.time(); first_swap_sol = first_improvement(machine_order_constr, processing_time, machines_required, n, m); first_swap_end_time = time.time()
  best_swap_start_time = time.time(); best_swap_sol = best_improvement(machine_order_constr, processing_time, machines_required, n, m); best_swap_end_time = time.time()

  first_insert_for_start_time = time.time(); first_insert_for_sol = first_improvement(machine_order_constr, processing_time, machines_required, n, m, insert_forward); first_insert_for_end_time = time.time()
  best_insert_for_start_time = time.time(); best_insert_for_sol = best_improvement(machine_order_constr, processing_time, machines_required, n, m, insert_forward); best_insert_for_end_time = time.time()

  first_insert_back_start_time = time.time(); first_insert_back_sol = first_improvement(machine_order_constr, processing_time, machines_required, n, m, insert_backward); first_insert_back_end_time = time.time()
  best_insert_back_start_time = time.time(); best_insert_back_sol = best_improvement(machine_order_constr, processing_time, machines_required, n, m, insert_backward); best_insert_back_end_time = time.time()

  

  print(file, "-> Constr: ", ans_constr, " Cota: ", cota_inferior)
  print( "first swap: ", makespan(first_swap_sol, processing_time, machines_required, n, m), "|",
         "first insert forward: ", makespan(first_insert_for_sol, processing_time, machines_required, n, m), "|" ,
         "first insert backward: ", makespan(first_insert_back_sol, processing_time, machines_required, n, m))
  print( "best swap: ", makespan(best_swap_sol, processing_time, machines_required, n, m), "|",
         "best insert forward: ", makespan(best_insert_for_sol, processing_time, machines_required, n, m), "|" ,
         "best insert backward: ", makespan(best_insert_back_sol, processing_time, machines_required, n, m))
  print(" RVNS -> " , makespan(rvns_sol, processing_time, machines_required, n, m))
  
  write_xlsx(first_swap_sol, makespan(first_swap_sol, processing_time, machines_required, n, m), first_swap_end_time - first_swap_start_time, file , "JSSP_coberndorm_first_improvement_swap.xlsx")
  write_xlsx(first_insert_for_sol, makespan(first_insert_for_sol, processing_time, machines_required, n, m), first_insert_for_end_time - first_insert_for_start_time, file, "JSSP_coberndorm_first_improvement_insert_forward.xlsx")
  write_xlsx(first_insert_back_sol, makespan(first_insert_back_sol, processing_time, machines_required, n, m), first_insert_back_end_time - first_insert_back_start_time, file, "JSSP_coberndorm_first_improvement_insert_backward.xlsx")

  write_xlsx(best_swap_sol, makespan(best_swap_sol, processing_time, machines_required, n, m), best_swap_end_time - best_swap_start_time, file , "JSSP_coberndorm_best_improvement_swap.xlsx")
  write_xlsx(best_insert_for_sol, makespan(best_insert_for_sol, processing_time, machines_required, n, m), best_insert_for_end_time - best_insert_for_start_time, file, "JSSP_coberndorm_best_improvement_insert_forward.xlsx")
  write_xlsx(best_insert_back_sol, makespan(best_insert_back_sol, processing_time, machines_required, n, m), best_insert_back_end_time - best_insert_back_start_time, file, "JSSP_coberndorm_best_improvement_insert_backward.xlsx")


JSSP1.txt -> Constr:  1462  Cota:  991
first swap:  1386 | first insert forward:  1397 | first insert backward:  1386
best swap:  1386 | best insert forward:  1400 | best insert backward:  1386
 RVNS ->  1417


In [6]:
with open('makespan_constr.txt', 'r') as f:
    makespan_constr = f.readlines()
    makespan_constr = [int(line.strip()) for line in makespan_constr]

In [17]:
import os
import xlrd

# Get a list of Excel files in the current directory
directory_files = os.listdir()
directory_files.sort()
directory_files = [file for file in directory_files if file.endswith('.xlsx')]

for file in directory_files:
    # Specify the path to your Excel file
    excel_file_path = file
    print("Current method: ", excel_file_path)
    average_distance_to_limit = []
    average_improvement = []

    for i in range(1,16+1):
        sheet_name = f"JSSP{i}.txt"  # Change this to the desired sheet name
        column_letter = "B"      # Change this to the desired column letter

        # Open the Excel workbook
        workbook = xlrd.open_workbook(excel_file_path, formatting_info=True)

        # Select the specific sheet by name
        sheet = workbook.sheet_by_name(sheet_name)

        # Find the last row with data in the specified column (column B)
        max_row = sheet.nrows -1

        # Read the value from the last cell in column B
        makespan_val = sheet.cell_value(max_row, 0)
        # Read the value from the last cell in column B
        time_val = sheet.cell_value(max_row, 1)

        cota_file = cotas[i-1]

        efficacy = (makespan_val - cota_file)/cota_file
        average_distance_to_limit.append(efficacy)
        average_improvement.append((makespan_constr[i-1] - makespan_val)/makespan_constr[i-1])

    print("average distance to limit: ", round(sum(average_distance_to_limit)/len(average_distance_to_limit), 4))
    print("average improvement: ", round(sum(average_improvement)/len(average_improvement),4))
    

Current method:  JSSP_coberndorm_best_improvement_insert_backward.xlsx
average distance to limit:  0.3155
average improvement:  0.0329
Current method:  JSSP_coberndorm_best_improvement_insert_forward.xlsx
average distance to limit:  0.3229
average improvement:  0.0281
Current method:  JSSP_coberndorm_best_improvement_swap.xlsx
average distance to limit:  0.3163
average improvement:  0.0323
Current method:  JSSP_coberndorm_first_improvement_insert_backward.xlsx
average distance to limit:  0.309
average improvement:  0.0376
Current method:  JSSP_coberndorm_first_improvement_insert_forward.xlsx
average distance to limit:  0.3231
average improvement:  0.0278
Current method:  JSSP_coberndorm_first_improvement_swap.xlsx
average distance to limit:  0.3137
average improvement:  0.0347
